In [6]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import os

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Sample data structure:
# text | label (0=real, 1=fake)

df = pd.read_csv("data/labeled/fake_news_dataset.csv")
print(df.head())

In [ ]:
# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Split into train/test
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="models/trained_bert",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
model.save_pretrained("models/bert_fake_news_classifier")
tokenizer.save_pretrained("models/bert_fake_news_classifier")

print("✅ Model saved to models/bert_fake_news_classifier/")

In [ ]:
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Show detailed classification report
logits, labels, _ = trainer.predict(tokenized_test)
predictions = torch.argmax(torch.tensor(logits), dim=1)

print("\n📊 Classification Report:")
print(classification_report(labels, predictions))